In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Generate Data

In [3]:
# Ensure the parent directory is in the Python path
import sys
import os
sys.path.append(os.path.abspath(".."))

from data import SyntheticDataGeneratorPlus

### RCT with treatment rate 0.5
with pd.HDFStore("RCT_0_5.h5") as store:
    for i in ['A', 'B', 'C', 'D', 'E']:
        dataset_name = f'RCT_0_5_scenario_{i}'
        gen = SyntheticDataGeneratorPlus(scenario=i, dataset_name=dataset_name,
                                         n_samples=50000, random_state=2025,
                                         RCT=True, treatment_proportion=0.5, unobserved=False, overlap=True)
        dsets = gen.generate_datasets() 
        store[f"scenario_{i}/data"] = dsets['data']
        store.get_storer(f"scenario_{i}/data").attrs.metadata = dsets['metadata']

### RCT with treatment rate 0.05
with pd.HDFStore("RCT_0_05.h5") as store:
    for i in ['A', 'B', 'C', 'D', 'E']:
        dataset_name = f'RCT_0_05_scenario_{i}'
        gen = SyntheticDataGeneratorPlus(scenario=i, dataset_name=dataset_name,
                                         n_samples=50000, random_state=2025,
                                         RCT=True, treatment_proportion=0.05, unobserved=False, overlap=True)
        dsets = gen.generate_datasets() 
        store[f"scenario_{i}/data"] = dsets['data']
        store.get_storer(f"scenario_{i}/data").attrs.metadata = dsets['metadata']

### non-RCT with ignorability and overlap held - propensity score e(X)
with pd.HDFStore("e_X.h5") as store:
    for i in ['A', 'B', 'C', 'D', 'E']:
        dataset_name = f'e_X_scenario_{i}'
        gen = SyntheticDataGeneratorPlus(scenario=i, dataset_name=dataset_name,
                                        n_samples=50000, random_state=2025,
                                         RCT=False, unobserved=False, overlap=True)
        dsets = gen.generate_datasets()
        store[f"scenario_{i}/data"] = dsets['data']
        store.get_storer(f"scenario_{i}/data").attrs.metadata = dsets['metadata']

### non-RCT with ignorability violated and overlap held - propensity score e(X, U)
with pd.HDFStore("e_X_U.h5") as store:
    for i in ['A', 'B', 'C', 'D', 'E']:
        dataset_name = f'e_X_U_scenario_{i}'
        gen = SyntheticDataGeneratorPlus(scenario=i, dataset_name=dataset_name,
                                         n_samples=50000, random_state=2025,
                                         RCT=False, unobserved=True, overlap=True)
        dsets = gen.generate_datasets()
        store[f"scenario_{i}/data"] = dsets['data']
        store.get_storer(f"scenario_{i}/data").attrs.metadata = dsets['metadata']

### non-RCT with ignorability held but overlap violated - propensity score e(X)_no_overlap
with pd.HDFStore("e_X_no_overlap.h5") as store:
    for i in ['A', 'B', 'C', 'D', 'E']:
        dataset_name = f'e_X_no_overlap_scenario_{i}'
        gen = SyntheticDataGeneratorPlus(scenario=i, dataset_name=dataset_name,
                                        n_samples=50000, random_state=2025,
                                         RCT=False, unobserved=False, overlap=False)
        dsets = gen.generate_datasets()
        store[f"scenario_{i}/data"] = dsets['data']
        store.get_storer(f"scenario_{i}/data").attrs.metadata = dsets['metadata']

### informative censoring
info_censor_baseline=0.1
info_censor_alpha=0.05

### informative_censoring and non-RCT with ignorability and overlap held - propensity score e(X)
with pd.HDFStore("e_X_info_censor.h5") as store:
    for i in ['A', 'B', 'C', 'D', 'E']:
        dataset_name = f'e_X_info_censor_scenario_{i}'
        gen = SyntheticDataGeneratorPlus(scenario=i, dataset_name=dataset_name,
                                         n_samples=50000, random_state=2025,
                                         informative_censoring=True, RCT=False, 
                                         unobserved=False, overlap=True,
                                         info_censor_baseline=info_censor_baseline,
                                         info_censor_alpha=info_censor_alpha)
        dsets = gen.generate_datasets() 
        store[f"scenario_{i}/data"] = dsets['data']
        store.get_storer(f"scenario_{i}/data").attrs.metadata = dsets['metadata']

### informative_censoring and non-RCT with ignorability violated and overlap held - propensity score e(X, U)
with pd.HDFStore("e_X_U_info_censor.h5") as store:
    for i in ['A', 'B', 'C', 'D', 'E']:
        dataset_name = f'e_X_U_info_censor_scenario_{i}'
        gen = SyntheticDataGeneratorPlus(scenario=i, dataset_name=dataset_name,
                                         n_samples=50000, random_state=2025,
                                         informative_censoring=True, RCT=False, 
                                         unobserved=True, overlap=True,
                                         info_censor_baseline=info_censor_baseline,
                                         info_censor_alpha=info_censor_alpha)
        dsets = gen.generate_datasets() 
        store[f"scenario_{i}/data"] = dsets['data']
        store.get_storer(f"scenario_{i}/data").attrs.metadata = dsets['metadata']

### informative_censoring and non-RCT with ignorability held but overlap violated - propensity score e(X)
with pd.HDFStore("e_X_no_overlap_info_censor.h5") as store:
    for i in ['A', 'B', 'C', 'D', 'E']:
        dataset_name = f'e_X_no_overlap_info_censor_scenario_{i}'
        gen = SyntheticDataGeneratorPlus(scenario=i, dataset_name=dataset_name,
                                         n_samples=50000, random_state=2025,
                                         informative_censoring=True, RCT=False, 
                                         unobserved=False, overlap=False,
                                         info_censor_baseline=info_censor_baseline,
                                         info_censor_alpha=info_censor_alpha)
        dsets = gen.generate_datasets() 
        store[f"scenario_{i}/data"] = dsets['data']
        store.get_storer(f"scenario_{i}/data").attrs.metadata = dsets['metadata']

# Data Characteristics

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

def load_scenario_data(h5_file_path, scenario_num):
    key = f"scenario_{scenario_num}/data"
    with pd.HDFStore(h5_file_path, mode='r') as store:
        if key not in store:
            return None  # Scenario not found
        df = store[key]
        metadata = store.get_storer(key).attrs.metadata
    return {"dataset": df, "metadata": metadata}

In [10]:
store_files = [
    "../synthetic_data/RCT_0_5.h5",
    "../synthetic_data/RCT_0_05.h5",
    "../synthetic_data/e_X.h5",
    "../synthetic_data/e_X_U.h5",
    "../synthetic_data/e_X_no_overlap.h5",
    "../synthetic_data/e_X_info_censor.h5",
    "../synthetic_data/e_X_U_info_censor.h5",
    "../synthetic_data/e_X_no_overlap_info_censor.h5"
]

experiment_setups = {}

for path in store_files:
    base_name = os.path.splitext(os.path.basename(path))[0]  # e.g. RCT_0_5
    scenario_dict = {}
    for scenario in ['A', 'B', 'C', 'D', 'E']:
        try:
            result = load_scenario_data(path, scenario)
            if result is not None:
                scenario_dict[f"scenario_{scenario}"] = result
        except Exception as e:
            # Log or ignore as needed
            print(f"Error loading scenario {scenario} from {path}: {e}")
            continue
    experiment_setups[base_name] = scenario_dict

In [11]:
summary_characteristics = {}

for setup_name, setup_dict in tqdm(experiment_setups.items(), desc="Experiment Setups"):
    summary_characteristics[setup_name] = {}
    for scenario_key in tqdm(setup_dict, desc=f"{setup_name} Scenarios", leave=False):
        dataset_df = setup_dict[scenario_key]["dataset"]
            
        # Store placeholder for later population
        cur_dataset_df = experiment_setups[setup_name][scenario_key]["dataset"]
        summary_characteristics[setup_name][scenario_key] = {'censoring_rate': 1-cur_dataset_df['event'].mean(),
                                                             'treatment_rate': cur_dataset_df['W'].mean(),
                                                             'event_time_min': cur_dataset_df['T'].min(),
                                                             'event_time_median': cur_dataset_df['T'].median(),
                                                             'event_time_max': cur_dataset_df['T'].max(),
                                                             'event_time_mean': cur_dataset_df['T'].mean(),
                                                             'event_time_std': cur_dataset_df['T'].std(),
                                                             'censoring_time_min': cur_dataset_df['C'].min(),
                                                             'censoring_time_median': cur_dataset_df['C'].median(),
                                                             'censoring_time_max': cur_dataset_df['C'].max(),
                                                             'censoring_time_mean': cur_dataset_df['C'].mean(),
                                                             'censoring_time_std': cur_dataset_df['C'].std(),
                                                             'ate': (cur_dataset_df['T1']-cur_dataset_df['T0']).mean(),
                                                             'cate_min': (cur_dataset_df['T1']-cur_dataset_df['T0']).min(),
                                                             'cate_median': (cur_dataset_df['T1']-cur_dataset_df['T0']).median(),
                                                             'cate_max': (cur_dataset_df['T1']-cur_dataset_df['T0']).max(),}
        

# Convert the summary_characteristics dictionary to a DataFrame
summary_df = pd.DataFrame.from_dict({(i, j): summary_characteristics[i][j] 
                                       for i in summary_characteristics.keys() 
                                       for j in summary_characteristics[i].keys()},
                                      orient='index').round(6)

summary_df

Experiment Setups: 100%|██████████| 8/8 [00:00<00:00, 50.30it/s]


censoring_rate  treatment_rate  \
RCT_0_5                    scenario_A         0.20318         0.50220   
                           scenario_B         0.07288         0.50220   
                           scenario_C         0.39204         0.50220   
                           scenario_D         0.91260         0.50220   
                           scenario_E         0.79418         0.50220   
RCT_0_05                   scenario_A         0.19976         0.04924   
                           scenario_B         0.03638         0.04924   
                           scenario_C         0.39036         0.04924   
                           scenario_D         0.88072         0.04924   
                           scenario_E         0.76988         0.04924   
e_X                        scenario_A         0.20140         0.50288   
                           scenario_B         0.06630         0.50288   
                           scenario_C         0.39266         0.50288   
                           scenario_D         0.91392         0.50288   
                           scenario_E         0.78930         0.50288   
e_X_U                      scenario_A         0.20064         0.53890   
                           scenario_B         0.07322         0.53890   
                           scenario_C         0.39226         0.53890   
                           scenario_D         0.91840         0.53890   
                           scenario_E         0.79530         0.53890   
e_X_no_overlap             scenario_A         0.20318         0.50004   
                           scenario_B         0.08156         0.50004   
                           scenario_C         0.39254         0.50004   
                           scenario_D         0.91192         0.50004   
                           scenario_E         0.80340         0.50004   
e_X_info_censor            scenario_A         0.11580         0.50288   
                           scenario_B         0.05156         0.50288   
                           scenario_C         0.88460         0.50288   
                           scenario_D         0.36574         0.50288   
                           scenario_E         0.92562         0.50288   
e_X_U_info_censor          scenario_A         0.11588         0.53890   
                           scenario_B         0.05432         0.53890   
                           scenario_C         0.88806         0.53890   
                           scenario_D         0.38074         0.53890   
                           scenario_E         0.92948         0.53890   
e_X_no_overlap_info_censor scenario_A         0.11630         0.50004   
                           scenario_B         0.05828         0.50004   
                           scenario_C         0.89098         0.50004   
                           scenario_D         0.40292         0.50004   
                           scenario_E         0.93154         0.50004   

                                       event_time_min  event_time_median  \
RCT_0_5                    scenario_A        0.000000           0.171960   
                           scenario_B        0.001912           0.207141   
                           scenario_C        0.000000           7.000000   
                           scenario_D        0.013666           1.672461   
                           scenario_E        0.000000           8.000000   
RCT_0_05                   scenario_A        0.000000           0.173173   
                           scenario_B        0.002733           0.187343   
                           scenario_C        0.000000           7.000000   
                           scenario_D        0.027282           1.628126   
                           scenario_E        0.000000           8.000000   
e_X                        scenario_A        0.000000           0.171088   
                           scenario_B        0.001912           0.202417   
                           scenario_C        0.000000    

In [12]:
TRUE_ATE_DICT = summary_df['ate'].to_dict()
TRUE_ATE_DICT

{('RCT_0_5', 'scenario_A'): 0.163441,
 ('RCT_0_5', 'scenario_B'): 0.124969,
 ('RCT_0_5', 'scenario_C'): 0.74996,
 ('RCT_0_5', 'scenario_D'): 0.723925,
 ('RCT_0_5', 'scenario_E'): 0.7537,
 ('RCT_0_05', 'scenario_A'): 0.163441,
 ('RCT_0_05', 'scenario_B'): 0.124969,
 ('RCT_0_05', 'scenario_C'): 0.74996,
 ('RCT_0_05', 'scenario_D'): 0.723925,
 ('RCT_0_05', 'scenario_E'): 0.7537,
 ('e_X', 'scenario_A'): 0.163441,
 ('e_X', 'scenario_B'): 0.124969,
 ('e_X', 'scenario_C'): 0.74996,
 ('e_X', 'scenario_D'): 0.723925,
 ('e_X', 'scenario_E'): 0.7537,
 ('e_X_U', 'scenario_A'): 0.003744,
 ('e_X_U', 'scenario_B'): 0.131728,
 ('e_X_U', 'scenario_C'): 0.74036,
 ('e_X_U', 'scenario_D'): 0.830668,
 ('e_X_U', 'scenario_E'): 0.74032,
 ('e_X_no_overlap', 'scenario_A'): 0.163441,
 ('e_X_no_overlap', 'scenario_B'): 0.124969,
 ('e_X_no_overlap', 'scenario_C'): 0.74996,
 ('e_X_no_overlap', 'scenario_D'): 0.723925,
 ('e_X_no_overlap', 'scenario_E'): 0.7537,
 ('e_X_info_censor', 'scenario_A'): 0.163441,
 ('e_X_i

In [ ]:
TRUE_ATE = {('RCT_0_5', 'scenario_B'): 0.124969, ('RCT_0_5', 'scenario_A'): 0.163441, ('RCT_0_5', 'scenario_C'): 0.74996,
            ('RCT_0_5', 'scenario_E'): 0.7537, ('RCT_0_5', 'scenario_D'): 0.723925,
            ('RCT_0_05', 'scenario_B'): 0.124969, ('RCT_0_05', 'scenario_A'): 0.163441, ('RCT_0_05', 'scenario_C'): 0.74996,
            ('RCT_0_05', 'scenario_E'): 0.7537, ('RCT_0_05', 'scenario_D'): 0.723925,
            ('e_X', 'scenario_B'): 0.124969, ('e_X', 'scenario_A'): 0.163441, ('e_X', 'scenario_C'): 0.74996,
            ('e_X', 'scenario_E'): 0.7537, ('e_X', 'scenario_D'): 0.723925,
            ('e_X_U', 'scenario_B'): 0.131728, ('e_X_U', 'scenario_A'): 0.003744, ('e_X_U', 'scenario_C'): 0.74036,
            ('e_X_U', 'scenario_E'): 0.74032, ('e_X_U', 'scenario_D'): 0.830668,
            ('e_X_no_overlap', 'scenario_B'): 0.124969, ('e_X_no_overlap', 'scenario_A'): 0.163441, ('e_X_no_overlap', 'scenario_C'): 0.74996,
            ('e_X_no_overlap', 'scenario_E'): 0.7537, ('e_X_no_overlap', 'scenario_D'): 0.723925,
            ('e_X_info_censor', 'scenario_B'): 0.124969, ('e_X_info_censor', 'scenario_A'): 0.163441, ('e_X_info_censor', 'scenario_C'): 0.74996,
            ('e_X_info_censor', 'scenario_E'): 0.7537, ('e_X_info_censor', 'scenario_D'): 0.723925,
            ('e_X_U_info_censor', 'scenario_B'): 0.131728, ('e_X_U_info_censor', 'scenario_A'): 0.003744, ('e_X_U_info_censor', 'scenario_C'): 0.74036,
            ('e_X_U_info_censor', 'scenario_E'): 0.74032, ('e_X_U_info_censor', 'scenario_D'): 0.830668,
            ('e_X_no_overlap_info_censor', 'scenario_B'): 0.124969, ('e_X_no_overlap_info_censor', 'scenario_A'): 0.163441, ('e_X_no_overlap_info_censor', 'scenario_C'): 0.74996,
            ('e_X_no_overlap_info_censor', 'scenario_E'): 0.7537, ('e_X_no_overlap_info_censor', 'scenario_D'): 0.723925}

In [9]:
TRUE_ATE.get((setup_name, scenario_key), 0)

0.723925